In [157]:
import json
import numpy as np
import os
import glob
import cv2

In [69]:
def anno_txt2dict(root, label_name):
    """
    从gt的txt文件中读取类别和bbox
    """
    anno_info = dict()
    bbox = []
    category = []
    with open(root + label_name, 'r') as f:
        annolist = f.readlines()
        for anno in annolist:
            s,x,y,w,h = anno.strip().split()
            category.append(int(s))
            bbox.append([float(x),float(y),float(w),float(h)])
    anno_info['category'] = category
    anno_info['bbox'] = bbox           
    return anno_info





def xyxy2XcYcwh(bboxes, image_height, image_width):
    """
    输入xyxy 格式的bbox numpy
    输出XcYcwh 格式的bbox numpy
    """
  
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
   
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    
    bboxes[..., [2, 3]] = bboxes[..., [2, 3]] - bboxes[..., [0, 1]]
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes


def xywh2xyxy(bboxes, image_height, image_width):
    """
    输入xywh 格式的bbox numpy
    输出xyxy 格式的bbox numpy
    """
    bboxes = bboxes.copy().astype(float)
    
    
#     bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
#     bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    #bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes


def XcYcwh2xyxy(bboxes, image_height, image_width):
    """
    输入XcYcwh 格式的bbox list
    输出xyxy 格式的bbox numpy
    """
    bboxes = np.asarray(bboxes).reshape(-1,4)
    bboxes = bboxes.copy().astype(int)
    
    
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes


def calc_iou(gt_bboxs, pred_bboxs):
    """
    计算pred 和各个gt bbox之间的iou, bbox是x1，y1，x2，y2格式
    gt_bboxs numpy (N, 4)
    pred_bboxs numpy (M, 4)
    
    返回iou （N，M）
    """
#     gt_bboxs = np.asarray(gt_bboxs).reshape(-1,4)
#     pred_bboxs = np.asarray(pred_bboxs).reshape(-1, 4)
#     print(gt_bboxs, pred_bboxs)
    
    gt_area = (gt_bboxs[:,2] - gt_bboxs[:,0]) * (gt_bboxs[:,3] - gt_bboxs[:,1])
    pred_area = (pred_bboxs[:,2] - pred_bboxs[:,0]) * (pred_bboxs[:,3] - pred_bboxs[:,1])

    
    width = np.maximum(np.minimum(gt_bboxs[:, 2], pred_bbox[:, 2]) - np.maximum(gt_bboxs[:, 0], pred_bbox[:, 0]) ,0.)
    height = np.maximum(np.minimum(gt_bboxs[:, 3], pred_bbox[:, 3]) - np.maximum(gt_bboxs[:, 1], pred_bbox[:, 1]) ,0.)
    
    
    inter = width * height
    
    return np.maximum(inter / (gt_area + pred_area - inter), np.zeros_like(inter))
    

In [320]:
root = './kfold/'
kfold_name = 'kfold_0/'
pred_json_name = 'results0.bbox.json'
preds = json.load(open(root + json_name, 'r'))
write_dir = root + 'new/'

In [321]:
###image_id和label名的对应####
label_dir = root + kfold_name + 'labels/val/'
label_names = os.listdir(label_dir)  #val.json中的图片也是从0-9而不是从小到大
label_names

['100.txt',
 '1008.txt',
 '1024.txt',
 '1032.txt',
 '1037.txt',
 '1040.txt',
 '1048.txt',
 '1049.txt',
 '1053.txt',
 '106.txt',
 '1069.txt',
 '1071.txt',
 '1073.txt',
 '1077.txt',
 '108.txt',
 '1103.txt',
 '1104.txt',
 '111.txt',
 '112.txt',
 '1120.txt',
 '1124.txt',
 '1144.txt',
 '1145.txt',
 '1166.txt',
 '1183.txt',
 '1191.txt',
 '1194.txt',
 '1201.txt',
 '1205.txt',
 '1208.txt',
 '1209.txt',
 '1211.txt',
 '1214.txt',
 '1217.txt',
 '1224.txt',
 '123.txt',
 '1232.txt',
 '1242.txt',
 '1245.txt',
 '1246.txt',
 '1251.txt',
 '1261.txt',
 '1264.txt',
 '1270.txt',
 '1274.txt',
 '1279.txt',
 '1291.txt',
 '1293.txt',
 '1294.txt',
 '1305.txt',
 '1314.txt',
 '1328.txt',
 '133.txt',
 '1330.txt',
 '1334.txt',
 '1335.txt',
 '1340.txt',
 '1345.txt',
 '1352.txt',
 '1357.txt',
 '1371.txt',
 '1374.txt',
 '1385.txt',
 '1397.txt',
 '1398.txt',
 '14.txt',
 '1402.txt',
 '1412.txt',
 '1413.txt',
 '1416.txt',
 '1420.txt',
 '1422.txt',
 '1423.txt',
 '1438.txt',
 '144.txt',
 '1441.txt',
 '1442.txt',
 '145.txt

In [382]:
###遍历pred的每个bbox####
score_thr = 0.8
for i, pred in enumerate(preds):
    image_id = int(pred['image_id'])
    pred_bbox = np.asarray(pred['bbox']).reshape(-1,4)
    score = pred['score']
    category = pred['category_id']
    
    
    if score < score_thr:  #只选择score大于0.7的框
        continue
    else:
        gt = anno_txt2dict(root + kfold_name + 'labels/val/',  label_names[image_id]) #对应图片的gt
        img_h, img_w = cv2.imread(root + kfold_name + 'images/val/' + label_names[image_id].replace('.txt', '.jpg')).shape[:2]
        #gt['bbox']  = XcYcwh2xyxy(gt['bbox'], img_h, img_w)  #转换bbox格式   
        
        
        gt_bbox  = XcYcwh2xyxy(gt['bbox'], img_h, img_w)  #转换成voc格式
        pred_bbox = xywh2xyxy(pred_bbox, img_h, img_w)  #转换成voc格式
        
        
        iou = calc_iou(gt_bbox, pred_bbox)
        idx = np.argmax(iou)  # 最大iou的索引
                    
        if category == gt['category'][idx]: #如果类别相同，
            
            if iou[idx] > 0.5: #最大iou >0.5 替换bbox标签               
                gt_bbox[idx] = pred_bbox
            else: #增加bbox标签
                gt_bbox = np.r_[gt_bbox, pred_bbox]
                gt['category'].append(category)
            
#         else: #如果类别不同，删除原有标签
#             gt_bbox = np.delete(gt_bbox, idx)

        if label_names[image_id] == '2019.txt':
            print(gt['bbox'])
            print(gt_bbox)
            print(pred_bbox)
#             print(iou)
#             print(idx)
            break
        else:
            continue
 
       
    ####写新的label txt文件####
    gt_category= gt['category']
    gt_bbox = xyxy2XcYcwh(gt_bbox, img_h, img_w).round(6).tolist() #转成yolo格式
 
    if not os.path.exists(write_dir):
        os.mkdir(write_dir)
    with open(write_dir + label_names[image_id], 'w') as f:
        for b in range(len(gt_bbox)):
            f.write(str(gt_category[b]) + ' ')
            bbox = str(gt_bbox[b]).strip('[').strip(']').replace(',','') + '\n'
            f.write(bbox)

[[0.31, 0.55, 0.353333, 0.66], [0.805, 0.763333, 0.0766667, 0.16]]
[[ 57.92629242  70.44425964 132.89645386 262.51217651]
 [229.999995   204.9999     253.000005   252.9999    ]]
[[ 57.92629242  70.44425964 132.89645386 262.51217651]]


In [335]:
preds

[{'image_id': 0,
  'bbox': [51.872310638427734,
   12.820042610168457,
   205.07373428344727,
   259.32842540740967],
  'score': 0.004718902986496687,
  'category_id': 0},
 {'image_id': 0,
  'bbox': [56.84711456298828,
   12.873929977416992,
   192.3751449584961,
   257.227388381958],
  'score': 0.9810062646865845,
  'category_id': 1},
 {'image_id': 0,
  'bbox': [62.831295013427734,
   13.222116470336914,
   173.60766983032227,
   255.02284812927246],
  'score': 0.09493878483772278,
  'category_id': 1},
 {'image_id': 0,
  'bbox': [52.692115783691406,
   13.401931762695312,
   204.20760345458984,
   259.1777801513672],
  'score': 0.010731860995292664,
  'category_id': 1},
 {'image_id': 0,
  'bbox': [79.57017517089844,
   12.48663330078125,
   147.63360595703125,
   231.6863250732422],
  'score': 0.001664956915192306,
  'category_id': 1},
 {'image_id': 0,
  'bbox': [51.872310638427734,
   12.820042610168457,
   205.07373428344727,
   259.32842540740967],
  'score': 0.009102588519454002,


####  基于yolo格式(cls c_x, c_y, d_w,d_h, conf)的

策略： 把训练集和验证集中，置信度大于阈值的预测结果，作为伪标签

步骤：

    1）读取预测结果的txt信息
    
    2) 筛除conf不符合的pred框
    
    3) 计算pred框与每个gt框的iou
    
    4) 若与任意一个框的iou 都为 0 ，则把这个框加入gt_bbox
    
    5) 生成新的label txt

In [1]:
import os 
import numpy as np
import glob
import cv2
import torch
import shutil

In [18]:
pred_path = './fruit/train/kfold_pred'  #路径最后不能有 /
img_path = './fruit/train/JPEGImages'
label_path = './fruit/train/labels'
new_label_path = './fruit/train/new_labels/'
conf_thres = 0.8


label_list = glob.glob(os.path.join(label_path, '*txt'))
img_list =  list(map(lambda x: x.replace(label_path, img_path).replace('.txt', '.jpg'), label_list))
pred_list =  list(map(lambda x: x.replace(label_path, pred_path), label_list))

In [19]:
def read_txt(file):  #读取yolo pred
    data = np.loadtxt(file)
    return data


In [20]:
def XcYcwh2xyxy(bboxes, image_height, image_width):
    """
    输入XcYcwh 格式的bbox list
    输出xyxy 格式的bbox numpy
    """
    bboxes = np.asarray(bboxes).reshape(-1,4)
    bboxes = bboxes.copy().astype(float)
    
    
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    bboxes = bboxes.copy().astype(np.int) #voc 要 int格式
    
    return bboxes


def xyxy2XcYcwh(bboxes, image_height, image_width):
    """
    输入xyxy 格式的bbox numpy
    输出XcYcwh 格式的bbox numpy
    """
  
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
   
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    
    bboxes[..., [2, 3]] = bboxes[..., [2, 3]] - bboxes[..., [0, 1]]
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

In [21]:
def box_area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def calc_iou(gt_bboxs, pred_bboxs):
    """
    计算pred 和各个gt bbox之间的iou, bbox是x1，y1，x2，y2格式
    gt_bboxs numpy (N, 4)
    pred_bboxs numpy (M, 4)
    返回iou （N，M）
    """

    gt_bboxs = torch.tensor(gt_bboxs, dtype = torch.float32)
    pred_bboxs = torch.tensor(pred_bboxs, dtype = torch.float32)

    
    (a1, a2), (b1, b2) = gt_bboxs[:, None].chunk(2,2), pred_bboxs.chunk(2,1)
    inter = (torch.min(a2,b2) - torch.max(a1, b1)).clamp(0).prod(2)
    
    
    iou  = inter / (box_area(gt_bboxs.T)[:, None] + box_area(pred_bboxs.T) - inter)

    
    return iou.numpy()

    

In [28]:
print(pred_file)
os.path.exists(pred_file)

./fruit/train/kfold_pred\shoot548_4.txt


True

In [30]:
#删除原始保存路径
if os.path.isdir(new_label_path): 
    shutil.rmtree(new_label_path)
os.makedirs(new_label_path, exist_ok=True)



for i ,pred_file in enumerate(pred_list):
    if not os.path.exists(pred_file):
        print(pred_file)
        shutil.copy(label_list[i], new_label_path)
        continue
    #print(pred_file)
    pred = read_txt(pred_file).reshape(-1, 6)
    img = cv2.imread(img_list[i])
    img_h, img_w = img.shape[:2]
    label = read_txt(label_list[i]).reshape(-1, 5)
    gt_cls = label[:,0] 
        
        
    pred = pred[pred[:,5] > conf_thres]  #筛除conf不符合的pred框
    
    pred_bbox = XcYcwh2xyxy(pred[:,1:5], img_h, img_w)  #转换成voc格式的bbox
    label_bbox = XcYcwh2xyxy(label[:,1:], img_h, img_w)  #转换成voc格式
    
    iou = calc_iou(label_bbox, pred_bbox)
    

    for idx, instance in enumerate(iou.T):
       
        if np.all(instance == 0):
            print(label_bbox)
            
            print(pred_file)
            label_bbox = np.vstack((label_bbox, pred_bbox[idx]))
            gt_cls = np.append(gt_cls, pred[idx, 0])
            
            print(label_bbox)
    
    #写入txt
    with open(pred_file.replace(pred_path,new_label_path), 'a') as f:
        for j in range(len(label_bbox)):
            f.write(str(int(gt_cls[j])) + ' ')
            bbox = str(xyxy2XcYcwh(label_bbox, img_h, img_w)[j]).strip('[').strip(']').replace(',','') + '\n'
            f.write(bbox)
 
    
   #break
    
    

./fruit/train/kfold_pred\shoot548_4.txt
./fruit/train/kfold_pred\shoot568_3.txt
